# Projet maison n° 3

In [137]:
# imports
import pandas as pd
import glob as glb

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [138]:
%%time
import numpy as np
import pandas as pd
import glob
import os

# names us
def df_names_us():
    path = 'names'
    allFiles = glob.glob(os.path.join(path, "*.txt"))
    np_array_list = []
    
    for file in allFiles:
        df = pd.read_csv(file, index_col=None, header=None)
        df.insert(0, 'year', file[-8:-4])
        np_array_list.append(df.to_numpy())

    comb_np_array = np.vstack(np_array_list)
    data = pd.DataFrame(comb_np_array)
    data.set_axis(['year', 'name', 'gender', 'births'],
                  axis='columns', inplace=True)
    data[['year','births']] = data[['year','births']].astype(int)
    return data

Wall time: 0 ns


In [354]:
%%time
data = df_names_us()
data.head()


Wall time: 4.49 s


,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


In [355]:
data.dtypes

year       int32
name      object
gender    object
births     int32
dtype: object

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [349]:
# names fr
def df_names_fr():

    df = pd.read_csv('nat2020.csv', sep=';')
    df.set_axis(['gender', 'name', 'year', 'births'],
                axis='columns', inplace=True)
    df = df[['year', 'name', 'gender', 'births']]
    df.dropna(inplace=True)  
    df.drop(df.loc[df['year'] == 'XXXX'].index, inplace=True)
    df.drop(df.loc[df['name'] == '_PRENOMS_RARES'].index, inplace=True)
    df[['births', 'year']] = df[['births', 'year']].astype(int)
    df['year'] = df['year'].astype(object)  
    df['gender'] = df['gender'].map({2: 'F', 1: 'M'})
# Seuls les prénoms de 2 caractères et plus sont conservés
    df.drop(df.loc[df['name'].str.len() < 2].index, inplace=True)
# Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
    df.sort_values(['year', 'gender', 'births', 'name'],
                   ascending=(True, True, False, True), inplace=True)
# La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
    df['name'] = df['name'].str.title()
# L'index du DataFrame doit aller de 0 à N-1
    df = df.reset_index(drop=True)
    return df

In [350]:
df = df_names_fr()
df.dtypes


year      object
name      object
gender    object
births     int32
dtype: object

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [341]:
# taux de change
def df_taux_change(devises):
    df = pd.read_csv('Webstat_Export_20210929.csv', sep=';', skiprows=lambda x: x in range(1, 6), na_values=['-', ''])
    df['Titre :'] = pd.to_datetime(df['Titre :'],format='%d/%m/%Y')
    df.set_index('Titre :', inplace=True)
    df.index.names = ['Date']
    df.columns=[value.split('(')[1][:-1] for value in df.columns]
    df.dropna(inplace=True)
    df = df.applymap(lambda x: str(x.replace(',','.'))).astype('float')
    df = df[devises]
    return df

In [342]:
df = df_taux_change(['CHF', 'GBP', 'USD'])
df

,CHF,GBP,USD
Date,,,
2006-12-29,1.6069,0.67150,1.3170
2006-12-28,1.6058,0.67115,1.3173
2006-12-27,1.6058,0.67085,1.3159
2006-12-22,1.6029,0.67150,1.3192
2006-12-21,1.6032,0.67160,1.3178
...,...,...,...
2000-07-25,1.5538,0.62010,0.9410
2000-07-24,1.5555,0.61510,0.9332
2000-07-21,1.5511,0.61850,0.9363


### Tests

In [343]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [344]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [345]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... FAIL
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

FAIL: test_df_names_fr (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-343-b976908de6f7>", line 20, in test_df_names_fr
    self.assertEqual(len(df), 630407)
AssertionError: 630406 != 630407

----------------------------------------------------------------------
Ran 3 tests in 5.388s

FAILED (failures=1)
